In [1]:
import torch
import json
from PIL import Image
from models.predict import Custom_AlexNet
from torchvision.transforms import Compose, Resize, ToTensor
from torch.nn.functional import softmax
import warnings
warnings.simplefilter("ignore", Warning)
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import os
from sklearn.metrics import confusion_matrix
from datetime import timedelta

In [2]:
class MyJP2Dataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        hmi = Image.open(img_path)

        if self.transform:
            image = self.transform(hmi)
            
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))
        y_class = self.annotations.iloc[index, 2]
        return (image, y_label, y_class, img_path)

    def __len__(self):
        return len(self.annotations)

In [3]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:1' if use_cuda else 'cpu')
# device = torch.device('cpu')
torch.backends.cudnn.benchmark = True
print(device)

cuda:1


In [4]:
# Load Data
datapath = '/data/hmi_jpgs_512/'
partition3_path_6 = 'labels_store/6_M_Partition3_.csv'
partition4_path_6 = 'labels_store/6_M_Partition4_.csv'
partition3_path_12 = 'labels_store/12_M_Partition3_.csv'
partition4_path_12 = 'labels_store/12_M_Partition4_.csv'
partition3_path_18 = 'labels_store/18_M_Partition3_.csv'
partition4_path_18 = 'labels_store/18_M_Partition4_.csv'
partition3_path_24 = 'labels_store/24_M_Partition3_.csv'
partition4_path_24 = 'labels_store/24_M_Partition4_.csv'
transformations = Compose([
    ToTensor()
])

part3_6 = MyJP2Dataset(csv_file = partition3_path_6, 
                             root_dir = datapath,
                             transform = transformations)
part4_6 = MyJP2Dataset(csv_file = partition4_path_6, 
                             root_dir = datapath,
                             transform = transformations)
part3_12 = MyJP2Dataset(csv_file = partition3_path_12, 
                             root_dir = datapath,
                             transform = transformations)
part4_12 = MyJP2Dataset(csv_file = partition4_path_12, 
                             root_dir = datapath,
                             transform = transformations)
part3_18 = MyJP2Dataset(csv_file = partition3_path_18, 
                             root_dir = datapath,
                             transform = transformations)
part4_18 = MyJP2Dataset(csv_file = partition4_path_18, 
                             root_dir = datapath,
                             transform = transformations)
part3_24 = MyJP2Dataset(csv_file = partition3_path_24, 
                             root_dir = datapath,
                             transform = transformations)
part4_24 = MyJP2Dataset(csv_file = partition4_path_24, 
                             root_dir = datapath,
                             transform = transformations)

In [5]:
part3_loader_6 = DataLoader(dataset=part3_6, batch_size=12, num_workers=4, shuffle=False)
part4_loader_6 = DataLoader(dataset=part4_6, batch_size=12, num_workers=4, shuffle=False)
part3_loader_12 = DataLoader(dataset=part3_12, batch_size=12, num_workers=4, shuffle=False)
part4_loader_12 = DataLoader(dataset=part4_12, batch_size=12, num_workers=4, shuffle=False)
part3_loader_18 = DataLoader(dataset=part3_18, batch_size=12, num_workers=4, shuffle=False)
part4_loader_18 = DataLoader(dataset=part4_18, batch_size=12, num_workers=4, shuffle=False)
part3_loader_24 = DataLoader(dataset=part3_24, batch_size=12, num_workers=4, shuffle=False)
part4_loader_24 = DataLoader(dataset=part4_24, batch_size=12, num_workers=4, shuffle=False)
CUDA_LAUNCH_BLOCKING=1

In [6]:
model_PATH1 = 'trained_models/six.pth'
model_PATH2 = 'trained_models/twelve.pth'
model_PATH3 = 'trained_models/eighteen.pth'
model_PATH4 = 'trained_models/twentyfour.pth'
weights1 = torch.load(model_PATH1, map_location=device)
weights2 = torch.load(model_PATH2, map_location=device)
weights3 = torch.load(model_PATH3, map_location=device)
weights4 = torch.load(model_PATH4, map_location=device)
test_model = Custom_AlexNet().to(device)


#Generalize this
# checkpoint = torch.load(PATH)


In [7]:
def sklearn_Compatible_preds_and_targets(model_prediction_list, model_target_list, model_path_list, model_class_list):
    y_pred_list = []
    preds = []
    target_list = []
    tgts = []
    path_list = []
    path = []
    class_list = []
    clss = []
    y_pred_list = [a.squeeze().tolist() for a in model_prediction_list]
    preds = [item for sublist in y_pred_list for item in sublist]
    target_list = [a.squeeze().tolist() for a in model_target_list]
    tgts = [item for sublist in target_list for item in sublist]
    path_list = [a for a in model_path_list]
    path = [item for sublist in path_list for item in sublist]
    class_list = [a for a in model_class_list]
    clss = [item for sublist in class_list for item in sublist]
    return preds,tgts, path, clss


def accuracy_score(prediction, target):
    TN, FP, FN, TP = confusion_matrix(target, prediction).ravel()
    print("TP: ", TP, "FP: ", FP, "TN: ", TN, "FN: ", FN)
    #TSS Computation also known as "recall"
    tp_rate = TP / float(TP + FN) if TP > 0 else 0  
    fp_rate = FP / float(FP + TN) if FP > 0 else 0
    TSS = tp_rate - fp_rate
    
    #HSS2 Computation
    N = TN + FP
    P = TP + FN
    HSS = (2 * (TP * TN - FN * FP)) / float((P * (FN + TN) + (TP + FP) * N))

    return TSS, HSS

In [8]:
def predict(checkpoint, test_loader, desc ):
    test_target_list=[]
    test_prediction_list=[]
    test_path_list = []
    test_class_list = []
    test_model.load_state_dict(checkpoint['model_state_dict'])
    test_model.eval()
    print('***********************', desc, '*************************')
    with torch.no_grad():
        for d, t, cls, path in test_loader:
            # Get data to cuda if possible
            d = d.to(device=device)
            t = t.to(device=device)
    #         pa = path.to(device=device)
            test_target_list.append(t)
            test_path_list.append(list(path))
            test_class_list.append(list(cls))
    #         print(list(path))
            # forward pass
            s = test_model(d)
            #print("scores", s)

            # validation batch loss and accuracy
    #         l = criterion(s, t)
            p = softmax(s,dim=1)
    #         print(p[:,1])
            test_prediction_list.append(p[:,1])
            # accumulating the val_loss and accuracy
    #         val_loss += l.item()
            #val_acc += acc.item()
            del d,t,s,p
    a, b, c, d = sklearn_Compatible_preds_and_targets(test_prediction_list, test_target_list, test_path_list, test_class_list)
    preds = [int(i >=0.5) for i in a]
    print(accuracy_score(preds, b))
    prob_list = pd.DataFrame(
    {'timestamp': c,
     'flare_prob': a,
     'target': b,
     'goes_class': d
    })

    print(prob_list['target'].value_counts())
    prob_list['timestamp'] = prob_list['timestamp'].apply(lambda row: row[35:-4])
    prob_list['timestamp'] = pd.to_datetime(prob_list['timestamp'], format='%Y.%m.%d_%H.%M.%S')
    return prob_list
        


In [9]:
res3_6 = predict(weights1, part3_loader_6, '6 hours Partition-3')
res4_6 = predict(weights1, part4_loader_6, '6 hours Partition-4')
res3_12 = predict(weights2, part3_loader_12, '12 hours Partition-3')
res4_12 = predict(weights2, part4_loader_12, '12 hours Partition-4')
res3_18 = predict(weights3, part3_loader_18, '18 hours Partition-3')
res4_18 = predict(weights3, part4_loader_18, '18 hours Partition-4')
res3_24 = predict(weights4, part3_loader_24, '24 hours Partition-3')
res4_24 = predict(weights4, part4_loader_24, '24 hours Partition-4')

*********************** 6 hours Partition-3 *************************
TP:  578 FP:  3724 TN:  12020 FN:  350
(0.38631027474067836, 0.1425112127188712)
0    15744
1      928
Name: target, dtype: int64
*********************** 6 hours Partition-4 *************************
TP:  832 FP:  4760 TN:  10955 FN:  175
(0.5233211616678795, 0.16715871303559052)
0    15715
1     1007
Name: target, dtype: int64
*********************** 12 hours Partition-3 *************************
TP:  1023 FP:  4021 TN:  11131 FN:  497
(0.4076488078697271, 0.19954200819950868)
0    15152
1     1520
Name: target, dtype: int64
*********************** 12 hours Partition-4 *************************
TP:  1475 FP:  5004 TN:  9996 FN:  247
(0.5229621370499419, 0.23528490862397564)
0    15000
1     1722
Name: target, dtype: int64
*********************** 18 hours Partition-3 *************************
TP:  814 FP:  1652 TN:  13031 FN:  1175
(0.2967398126185199, 0.2688673488085104)
0    14683
1     1989
Name: target, dtype: in

In [10]:
res3_6.to_csv(r'results/res3_6.csv', index=False, header=True, columns=['timestamp', 'flare_prob', 'target', 'goes_class'])
res4_6.to_csv(r'results/res4_6.csv', index=False, header=True, columns=['timestamp', 'flare_prob', 'target', 'goes_class'])
res3_12.to_csv(r'results/res3_12.csv', index=False, header=True, columns=['timestamp', 'flare_prob', 'target', 'goes_class'])
res4_12.to_csv(r'results/res4_12.csv', index=False, header=True, columns=['timestamp', 'flare_prob', 'target', 'goes_class'])
res3_18.to_csv(r'results/res3_18.csv', index=False, header=True, columns=['timestamp', 'flare_prob', 'target', 'goes_class'])
res4_18.to_csv(r'results/res4_18.csv', index=False, header=True, columns=['timestamp', 'flare_prob', 'target', 'goes_class'])
res3_24.to_csv(r'results/res3_24.csv', index=False, header=True, columns=['timestamp', 'flare_prob', 'target', 'goes_class'])
res4_24.to_csv(r'results/res4_24.csv', index=False, header=True, columns=['timestamp', 'flare_prob', 'target', 'goes_class'])

In [11]:
def averaging(df):
    lis=[]
    for i in range(len(df)):
        sel = df[((df.timestamp<=df.timestamp[i]) & (df.timestamp>(df.timestamp[i]-timedelta(hours=24))))]
        avg = sel['flare_prob'].mean()
        lis.append([str(df.timestamp[i]), avg, df.target[i]])
    df_result = pd.DataFrame(lis, columns=['time', 'prob', 'tar'])
    return df_result
    

def max_voting(df):
    lis=[]
    for i in range(len(df)):
        sel = df[((df.timestamp<=df.timestamp[i]) & (df.timestamp>(df.timestamp[i]-timedelta(hours=24))))]
        t = threshold(sel['flare_prob'])
        vals,counts = np.unique(t, return_counts=True)
        index = np.argmax(counts)
        lis.append([str(df.timestamp[i]), vals[index], df.target[i]])
    df_result = pd.DataFrame(lis, columns=['time', 'prob', 'tar'])
    return df_result
    

def weighted(df):
    lis=[]
#     weights = np.array([0.025, 0.025, 0.05, 0.05, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.15])
    weights = np.array([0.025, 0.026, 0.027, 0.0285, 0.0295, 0.0305])

    for i in range(len(df)):
        sel = df[((df.timestamp<=df.timestamp[i]) & (df.timestamp>(df.timestamp[i]-timedelta(hours=24))))]
        if sel['flare_prob'].values.size<6:
            prob = sel['flare_prob'].mean()
        else:
            temp = np.multiply(sel['flare_prob'].values, weights)
            prob = np.mean(temp)
        lis.append([str(df.timestamp[i]), prob, df.target[i]])
    df_result = pd.DataFrame(lis, columns=['time', 'prob', 'tar'])
    return df_result

def threshold(df):
    y_pred = df.to_numpy().reshape(len(df),)
    yp = np.where(y_pred >= 0.5, 1, 0)
    return yp

In [12]:
def compare_sequential(fold, func, desc):
    pred = func(fold)
    zero_ones = threshold(pred['prob'])
    TSS, HSS = accuracy_score(zero_ones, pred['tar'].to_numpy().reshape(len(pred['tar']),))
    print('************************', desc, '***************************')
    print('TSS: {:.4f} | HSS: {:.4f}'.format(TSS, HSS))
    print('\n\n')


In [13]:
# #Averaging
compare_sequential(res3_6, averaging, 'Averaging on Partition-3  - 6 hours pred window')
compare_sequential(res4_6, averaging, 'Averaging on Partition-4  - 6 hours pred window')
compare_sequential(res3_12, averaging, 'Averaging on Partition-3  - 12 hours pred window')
compare_sequential(res4_12, averaging, 'Averaging on Partition-4  - 12 hours pred window')
compare_sequential(res3_18, averaging, 'Averaging on Partition-3  - 18 hours pred window')
compare_sequential(res4_18, averaging, 'Averaging on Partition-4  - 18 hours pred window')
compare_sequential(res3_24, averaging, 'Averaging on Partition-3  - 24 hours pred window')
compare_sequential(res4_24, averaging, 'Averaging on Partition-4  - 24 hours pred window')

#Max Voting
compare_sequential(res3_6, averaging, 'Majority Voting On Partition-3  - 6 hours pred window')
compare_sequential(res4_6, averaging, 'Majority Voting On Partition-4  - 6 hours pred window')
compare_sequential(res3_12, averaging, 'Majority Voting On Partition-3  - 12 hours pred window')
compare_sequential(res4_12, averaging, 'Majority Voting On Partition-4  - 12 hours pred window')
compare_sequential(res3_18, averaging, 'Majority Voting On Partition-3  - 18 hours pred window')
compare_sequential(res4_18, averaging, 'Majority Voting On Partition-4  - 18 hours pred window')
compare_sequential(res3_24, averaging, 'Majority Voting On Partition-3  - 24 hours pred window')
compare_sequential(res4_24, averaging, 'Majority Voting On Partition-4  - 24 hours pred window')
# #Weighted
# compare_sequential(fold1, weighted, 'weighted Fold-1')
# compare_sequential(fold2, weighted, 'weighted Fold-2')
# compare_sequential(fold3, weighted, 'weighted Fold-3')
# compare_sequential(fold4, weighted, 'weighted Fold-4')

TP:  587 FP:  3653 TN:  12091 FN:  341
************************ Averaging on Partition-3  - 6 hours pred window ***************************
TSS: 0.4005 | HSS: 0.1495



TP:  824 FP:  4864 TN:  10851 FN:  183
************************ Averaging on Partition-4  - 6 hours pred window ***************************
TSS: 0.5088 | HSS: 0.1602



TP:  1072 FP:  3951 TN:  11201 FN:  448
************************ Averaging on Partition-3  - 12 hours pred window ***************************
TSS: 0.4445 | HSS: 0.2182



TP:  1506 FP:  5138 TN:  9862 FN:  216
************************ Averaging on Partition-4  - 12 hours pred window ***************************
TSS: 0.5320 | HSS: 0.2349



TP:  821 FP:  1503 TN:  13180 FN:  1168
************************ Averaging on Partition-3  - 18 hours pred window ***************************
TSS: 0.3104 | HSS: 0.2893



TP:  1301 FP:  2332 TN:  12137 FN:  952
************************ Averaging on Partition-4  - 18 hours pred window ***************************
TSS: 0.4